# Hente og sende data på Internett

Python er mye brukt for å lage web applikasjoner men også for å utveklse data på internett.<br>
På nettet er det en uendelighet av dataportaler. Det kan være værdata, børskurser, statistikk, kartdata og register på det alle meste.

Data fra dataportaler kommer som oftest i [JSON](https://www.json.org/) eller [XML](https://no.wikipedia.org/wiki/XML) format.

## JSON Data fra webservice

JSON formatet er nesten helt likt en Python Dict, det er derfor ekstra lett å arbeide med.

Her er et eksempel på hvordan vi veldig enkelt kan hente kursen på dollar fra [Floatrates](http://www.floatrates.com/)

In [ ]:
import requests
r = requests.get('http://www.floatrates.com/daily/usd.json')
print('1 USD er', r.json()['nok']['rate'], 'NOK') 

*Oppgave: .json() funksjonen konverterer JSON data vi mottok til en dict. I eksempelet bruker jeg indeksen 'nok' og deretter 'rate' i denne ordlisten for å hente ut for norske kroner. List ut alle valuta som kommer i ordlisten. Hint: Prøv deg frem ved å bare printe .json() og .json['nok']*

### Åpne offentlige data i Norge

Det offentlige i Norge har mye åpne data tilgjengelig. En kan finne mye på [data.norge.no](http://data.norge.no/)<br>Telemark Fylkeskommune har også en del åpne data vi kan benytte oss av [her](http://apps.t-fk.no/api/#/datakilder). Et av disse er ansattregisteret som en kan se på Skogmo under [Personalet](http://www.skogmo.vgs.no/Om-skolen/Personalet).

In [ ]:
# Vi må først finne ut avdelingsnummeret til Skogmo Videregående
import requests
r = requests.get('http://ws.t-fk.no/?resource=departments&search=all&format=json')
departments = r.json()
#print(data)
for department in departments['results']:
    print(department)

Ved å studere resultate over, ser vi at Skogmo Videregående har departementId: 45 og vi kan hente alle ansatte som tilhører avdeling 45.

In [ ]:
import requests
r = requests.get('http://ws.t-fk.no/?resource=persons&search=departmentid&string=45&format=json')
persons = r.json()
#print(data)
for person in persons['results']:
    print(person)

*Oppgave: Skriv ut navn og e-post adresse på de ansatte basert på listen.*

Ved å studere api'et, så ser en at det kan brukes til å søke på navn på alle ansatte i fylkeskommunen. 

In [ ]:
import requests
r = requests.get('http://ws.t-fk.no/?resource=persons&search=fullname&string=Hogstad&format=json')
data = r.json()
print(data)


*Oppgave: Bruk input funksjonen for å lage et søk på ansatte i Telemark Fylkeskommune*

## Webskraping

[Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) er et Pyton bibliotek for å trekke data ut av HTML og XML.
Vi kan f.eks. trekke ut de siste nyhetssakene fra nrk.no med koden under.
<img  src="https://notebooks.azure.com/sauce1971/libraries/vgdatapythonmaster/raw/img%2Fnrk_nyheter.png">

In [ ]:
import requests
from bs4 import BeautifulSoup
r = requests.get('https://www.nrk.no/nyheter/')
soup = BeautifulSoup(r.text, 'html.parser')

for bulletin_title in soup.find_all('h2', class_= 'bulletin-title'): # Finner alle HTML h2 elementer med classen bulletin-title
    print(bulletin_title.text)

Beautiful Soup lar deg navigere rundt i XML/HTML Treet. I eksempelet nedenfor plukker vi også opp tidspunktet for nyhetssaken.

In [ ]:
import requests
from bs4 import BeautifulSoup
r = requests.get('https://www.nrk.no/nyheter/')
soup = BeautifulSoup(r.text, 'html.parser')
for bulletin_title in soup.find_all('h2', class_= 'bulletin-title'):
    title = bulletin_title.text
    # Vi navigerer til forelder noden og bruker find for å finne elementet med classen bulletin-publish-time
    publish_time = bulletin_title.parent.find(class_='bulletin-publish-time').text 
    print(publish_time, title)

*Oppgave: Skriv også ut de 100 første bokstavene av selve teksten til artikkelen.*

## Data i XML format

### api.met.no

Meteorolgisk institutt lar deg bruke det meste av sine [data helt gratis](https://api.met.no/). Akkurat på samme måte som de blir brukt på yr.no eller i yr appen. Mye av disse dataene er foreløpig bare i XML format.

[Nowcast](https://api.met.no/weatherapi/nowcast/0.9/documentation) Gir deg regn for de nærmeste timene.

In [ ]:
import requests
from bs4 import BeautifulSoup
r = requests.get('https://api.met.no/weatherapi/nowcast/0.9/?lat=59.173551&lon=9.620759')
soup = BeautifulSoup(r.text, 'xml')
for data in soup.find_all('time'):
    from_time = data['from'][11:16]
    precipitation = data.find('precipitation')['value']
    print(from_time, precipitation, 'mm')

*Oppgave: Analyser dataene fra Nowcast og gi et varsel om det snart blir regn eller ikke.*

*Oppgave: Fortell hvor lenge det er til det eventuelt kommer noe regn. Hint: Du må bruke [datetime.strptime()](https://docs.python.org/3/library/datetime.html?highlight=datetime#datetime.datetime.strptime) for å gjøre from_time om til et datetime objekt. (Google ekspempler på bruk av strptime).*

*Oppgave: Lag et stolpediagram som viser nedbør de nærmeste timene. Kan f.eks. lese [denne](https://pythonprogramming.net/bar-chart-histogram-matplotlib-tutorial/) for å se hvordan.*

### yr.no

En kan også få data som brukes på yr.no i xml ved å legge på varsel.xml i url'en<br>
https://www.yr.no/place/Norway/Telemark/Skien/Skien/varsel.xml

Tekstvarselet for i dag blir.

In [ ]:
import requests
from bs4 import BeautifulSoup
r = requests.get('https://www.yr.no/place/Norway/Telemark/Skien/Skien/varsel.xml')
soup = BeautifulSoup(r.text, 'xml')
text_forecast = soup.select('forecast text') # Returnerere en liste med matchende elementer
print(text_forecast[0].find('title').text)
print(text_forecast[0].find('body').text)

Nå verdiene er (Om du lager værstasjon uten sensorer?)

In [ ]:
import requests
from bs4 import BeautifulSoup
r = requests.get('https://www.yr.no/place/Norway/Telemark/Skien/Skien/varsel.xml')
soup = BeautifulSoup(r.text, 'xml')
weatherstation_name = soup.select('weatherstation')[0]['name']
weatherstation_temperature = soup.select('weatherstation')[0].find('temperature')['value']
print(f'{weatherstation_name}: {weatherstation_temperature} °C')


*Oppgave: Vis temperatur og navn på alle de næreste stasjonene. Der det er vind, så viser du også verdien for dette.*

*Oppgave: Lag en graf som viser temperatur fra varselet under "tabluar". (Dette er beskrevet i kapittel 4)* 

In [ ]:
import requests
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
r = requests.get('https://www.yr.no/place/Norway/Telemark/Skien/Skien/varsel.xml')
soup = BeautifulSoup(r.text, 'xml')
# Finner time taggene under tabular og plukker ut from attributten
time_data = [t['from'] for t in soup.find('tabular').find_all('time')] 
# Finner precipitation taggene under tabular og plukker ut value attributten
precipation_data = [p['value'] for p in soup.find('tabular').find_all('precipitation')]
x = [n for n in range(len(precipation_data))] # Lager x verdier fra 0 til antall datapunkt
plt.figure(figsize=(20,3)) # Angir størrelsen på figuren (Plottet)
plt.bar(x ,precipation_data, label="Nedbør", color='b')
#plt.xticks(x, time_data, rotation=45) # Tekster på X aksen
#plt.legend()
#plt.xlabel('Tid')
#plt.ylabel('Nedbør mm/t')
#plt.title('Nedbørsoversikt')
plt.show()
